In [44]:
#  CAO Points Analysis

# http://www2.cao.ie/points/l8.php

In [45]:
# Regular expressions 
import re

# Convenient HTTP requests
import requests as rq

# Dates and times
import datetime as dt

# Import Data frames
import pandas as pd

# For downloading.
import urllib.request as urlrq

# Import tabula to read table in pdf
import tabula as tb

# 2021 CAO Points

http://www2.cao.ie/points/l8.php

In [46]:
# Fetch the CAO points URL
resp = rq.get('http://www2.cao.ie/points/l8.php')

# Check connection '<Response [200]>' means OK
resp

# Check text
# resp.text

<Response [200]>

<br>

Save orginal data set
***

In [47]:
# Get the current date and time
now = dt.datetime.now()

# format as a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

print(nowstr)

20211205_225924


In [48]:
# Create a file path for the original data.
path = 'data/cao2021_' + nowstr + '.html'

<br>

Error on server
****
Technically, the server says we should decode as per:

    Content-Type: text/html; charset=iso-8859-1

However, one line uses \x96 which isn't defined in iso-8859-1.

Therefore we use the similar decoding standard cp1252, which is very similar but includes #x96.

In [49]:
# The server uses the wrong encoding, fix it.
original_encoding = resp.encoding

# Change to cp1252.
resp.encoding = 'cp1252'

In [50]:
# Save the original html file.
with open(path, 'w') as f:
    f.write(resp.text)

<br>

# Use regular expressions to select lines required
***

# Online Web tools used

Test regex on 
https://pythex.org/

Compare output of files using notepad ++ plugin
http://www.technicaloverload.com/compare-two-files-using-notepad/

In [51]:
# Compile the regular expression for matching lines with courses
# Documentation for re (regular expression)
# https://docs.python.org/3/library/re.html

# Copy of cao.csv orginal copy shows circa 949 lines

# Test of orginal re = 949
#re_course  = re.compile('([A-Z]{2}[0-9]{3})  .*([0-9]{3})')
#re_course  = re.compile('(\w{2}\d{3})\s{2}.*([0-9]{3})')
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

# Test of new re = 922
# re_course = re.compile(r'([A-Z]{2}[0-9]{3})  .*(\d{3}|[AQA]) *')

# Test of re = 949
# re_course  = re.compile('([A-Z]{2}[0-9]{3}).*(\#?|[0-9]{3}|[AQA]|\*?)')

# Test of re = 836
#re_course  = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)(\#?|[0-9]{3}|[AQA]\*?) *')

# Other test regex
# re_course  = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)\s+(\#?([0-9]{3})\*?).*')
# re_course  = re.compile(r'([A-Z]{2}[0-9]{3})\s+(.*)\s+(\#?[0-9]{3}\*?)\s+(\#?[0-9]{3}\*?)')

<br>

Loop throught the lines of response content from CAO

Copy of cao.csv orginal copy shows circa 949 lines

***

In [52]:
# The file path for the csv file.
path = 'data/cao2021_csv_' + nowstr + '.csv'

# Keep track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path, 'w') as f:
    # Loop through lines of the response.
    for line in resp.iter_lines():
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57]
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


# 2020 Points

https://www.cao.ie/index.php?page=points&p=2020

In [53]:
# Save file path for the orginal data
path = 'data/cao2020_' + nowstr + '.xlsx'

In [54]:
# get data from cao and save to path
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', path)


('data/cao2020_20211205_225924.xlsx',
 <http.client.HTTPMessage at 0x1eb9bc42880>)

## Load the spreadsheet using pandas

***

In [55]:
# Download and parse the excel spreadsheet.
df = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows=10)

In [56]:
# Spot check on the excel file
df.iloc[800]

CATEGORY (i.e.ISCED description)                       Law
COURSE TITLE                            Law and Accounting
COURSE CODE2                                         LM020
R1 POINTS                                              465
R1 Random *                                            NaN
R2 POINTS                                              NaN
R2 Random*                                             NaN
EOS                                                    465
EOS Random *                                           NaN
EOS Mid-point                                          499
LEVEL                                                    8
HEI                                 University of Limerick
Test/Interview #                                       NaN
avp                                                    NaN
v                                                      NaN
Column1                                                NaN
Column2                                                N

In [57]:
# Spot check the last row.
df.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [58]:
# first five rows of dataframe
df.iloc[0:5]

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# 1st, 4th, 7th, 25th row + 1st 8th 10th columns.
df.iloc[[0,3,6,24], [0,7,9]] 

,CATEGORY (i.e.ISCED description),EOS,EOS Mid-point
0,Business and administration,209,280
3,Arts,#+matric,#+matric
6,Arts,#+matric,#+matric
24,Information and Communication Technologies (ICTs),216,337


In [60]:
# Create a file path for the pandas data.
path = 'data/cao2020_' + nowstr + '.csv'

In [61]:
# Save pandas data frame to disk.
df.to_csv(path)

# 2019 Points

https://www.cao.ie/index.php?page=points&p=2019

***

In [62]:
# Save file path for the orginal data
path = 'data/cao2019_' + nowstr + '.pdf'

# get data from cao and save to path
urlrq.urlretrieve('http://www2.cao.ie/points/lvl8_19.pdf', path)


#df2019 = pd.read_csv('data/cao2019_20211029_113930_edited.csv', sep='\t')

('data/cao2019_20211205_225924.pdf',
 <http.client.HTTPMessage at 0x1eb9bc6adf0>)

## Load the pdf using pandas

***

In [63]:
# Download and parse the pdf with tabula module
# 
# import module tabula
# Documentation
# https://pypi.org/project/tabula-py/
df = tb.read_pdf('http://www2.cao.ie/points/lvl8_19.pdf', pages='all')

df

[   Course Code                             INSTITUTION and COURSE   EOS    Mid
 0          NaN                    Athlone Institute of Technology   NaN    NaN
 1        AL801    Software Design with Virtual Reality and Gaming   304  328.0
 2        AL802               Software Design with Cloud Computing   301  306.0
 3        AL803  Software Design with Mobile Apps and Connected...   309  337.0
 4        AL805        Network Management and Cloud Infrastructure   329  442.0
 5        AL810                                 Quantity Surveying   307  349.0
 6        AL820                 Mechanical and Polymer Engineering   300  358.0
 7        AL830                                    General Nursing   410  429.0
 8        AL832                                Psychiatric Nursing   387  403.0
 9        AL836                       Nutrition and Health Science   352  383.0
 10       AL837            Sports Science with Exercise Physiology   351  392.0
 11       AL838                         

In [64]:
# Create csv path
csvpath = 'data/cao2019_' + nowstr + '.csv'

# Convert pdf to csv
tb.convert_into( path , csvpath, output_format="csv", pages='all')

## Read csv into Pandas Dataframe

Clean data and remove colleges

- There is [965 rows x 4 columns] in the orignal dataframe.
- There is 35 Colleges
- All Colleges have NaN included
- Remove all colleges from dataframe 

In [65]:
# Read csv as a pandas dataframe
df2019 = pd.read_csv(csvpath, sep=',')

print(df2019)

    Course Code                             INSTITUTION and COURSE  EOS  Mid
0           NaN                    Athlone Institute of Technology  NaN  NaN
1         AL801    Software Design with Virtual Reality and Gaming  304  328
2         AL802               Software Design with Cloud Computing  301  306
3         AL803  Software Design with Mobile Apps and Connected...  309  337
4         AL805        Network Management and Cloud Infrastructure  329  442
..          ...                                                ...  ...  ...
960       WD200                                     Arts (options)  221  296
961       WD210                       Software Systems Development  271  329
962       WD211                                 Creative Computing  275  322
963       WD212                    Recreation and Sport Management  274  311
964       WD230           Mechanical and Manufacturing Engineering  273  348

[965 rows x 4 columns]


In [66]:
# All row with colleges have NAN 
# Find all rows that have NAN as a value
# https://www.geeksforgeeks.org/working-with-missing-data-in-pandas/

# creating bool series True for NaN values 
bool_series = pd.isnull(df2019["Course Code"]) 
    
# filtering data 
# displaying data only with Course Code = NaN 
df2019[bool_series]


,Course Code,INSTITUTION and COURSE,EOS,Mid
0,NaN,Athlone Institute of Technology,NaN,NaN
28,NaN,"Institute of Technology, Carlow",NaN,NaN
62,NaN,"Carlow College, St. Patrick`s",NaN,NaN
66,NaN,Cork Institute of Technology,NaN,NaN
110,NaN,University College Cork (NUI),NaN,NaN
173,NaN,American College,NaN,NaN
176,NaN,CCT College Dublin,NaN,NaN
178,NaN,Marino Institute of Education,NaN,NaN
185,NaN,Dublin Business School,NaN,NaN
209,NaN,Dublin City University,NaN,NaN


In [67]:
# Count how many rows have NaN
# https://datatofish.com/count-nan-pandas-dataframe/
count_nan = df2019['Course Code'].isna().sum()

print ('Count of NaN: ' + str(count_nan))

Count of NaN: 35


In [68]:
# Drop rows with NAN
# https://www.kite.com/python/answers/how-to-drop-empty-rows-from-a-pandas-dataframe-in-python

df2019.dropna(subset = ['Course Code'], inplace=True)

print(df2019)

    Course Code                             INSTITUTION and COURSE  EOS  Mid
1         AL801    Software Design with Virtual Reality and Gaming  304  328
2         AL802               Software Design with Cloud Computing  301  306
3         AL803  Software Design with Mobile Apps and Connected...  309  337
4         AL805        Network Management and Cloud Infrastructure  329  442
5         AL810                                 Quantity Surveying  307  349
..          ...                                                ...  ...  ...
960       WD200                                     Arts (options)  221  296
961       WD210                       Software Systems Development  271  329
962       WD211                                 Creative Computing  275  322
963       WD212                    Recreation and Sport Management  274  311
964       WD230           Mechanical and Manufacturing Engineering  273  348

[930 rows x 4 columns]


# End

In [69]:
df = tb.read_pdf('http://www2.cao.ie/points/lvl8_10.pdf', pages='all')

df

[      Unnamed: 0                                ADMISSION DATA 2010  \
 0            NaN                                                NaN   
 1            NaN                                            Level 8   
 2            NaN                                                NaN   
 3            NaN  The details  given are for general information...   
 4            NaN                                                NaN   
 5            NaN                                                NaN   
 6            NaN                                                NaN   
 7            NaN                                                NaN   
 8              *   Not all on this points score were offered places   
 9             **  Matriculated candidates are considered but adm...   
 10           NaN  the basis of performance in the music test and...   
 11           ***  Applicants are ranked as for other courses but...   
 12           NaN      decision depends on performance in interv